<a href="https://colab.research.google.com/github/shinchan75034/TFE_BigQuery/blob/master/How_TensorFlow_reads_BigQuery_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigQuery as data source for TensorFlow

This quick demo demonstrate how to use TensorFlow to access data in BigQuery. The objective here is to read structured data from a data warehouse such as BigQuery, and with `TensorFlow IO`, you can have the data ready as an iterator, which is ready for TensorFlow consumption, i.e., using it as a training dataset for your model.

### Requirement


1.   You need a project created under your GCP account. Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create one if you don't have one. Keep project name and project ID in a note pad.
2.   Also need to [enable billing](https://support.google.com/cloud/answer/6293499#enable-billing).
3.   Finally you need to [enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) API


Next, lets install `TensorFlow IO`. This is not preinstalled in `Colab`.

In [1]:
!pip install tensorflow-io

     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 421.8MB 24kB/s 
     |████████████████████████████████| 450kB 50.4MB/s 
     |████████████████████████████████| 3.9MB 50.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=bffd6b93442568c5bc94f8fe150c4787b772d330f9ed5167ec214c63ac1b24a6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully un

### Authenticate your credential

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Specify my project ID:

In [3]:
PROJECT_ID = "project1-190517" #@param {type:"string"}
! gcloud config set project $PROJECT_ID
%env GCLOUD_PROJECT=$PROJECT_ID

Updated property [core/project].
env: GCLOUD_PROJECT=project1-190517


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [0]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [5]:
# Display query output immediately

%%bigquery --project project1-190517
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


You can also designate the output to a `pandas` dataframe:

In [0]:
# Save output in a variable `df`

%%bigquery --project project1-190517 df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [7]:
df

,total_rows
0,114420316


In [8]:
type(df)

pandas.core.frame.DataFrame

# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [0]:
project_id = 'project1-190517'

### Sample approximately 2000 random rows

In [10]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### Describe the sampled data

In [11]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,2067.000000,2067.000000,2067.000000,2067.000000,2067.000000,2067.000000,2067.000000,1958.000000,1958.000000,1561.000000,1561.000000,758.000000,758.000000,1848.000000,1848.000000,2038.000000,2038.000000,1999.000000,268.000000,2062.000000,1906.000000,124.000000
mean,498485.054185,90808.979681,1986.985970,6.519594,15.804064,51.793904,12.846638,41.274259,12.799796,1015.047918,11.226778,965.651584,11.915567,11.869102,12.513528,6.731845,12.834642,12.021311,25.450000,43.258729,0.078736,10.352419
std,298966.205193,25632.352131,16.102386,3.478182,8.714678,23.750090,7.885200,22.005168,7.927608,9.778742,7.511859,70.346762,7.856423,8.564988,7.816648,4.819194,7.868746,6.590083,8.039623,23.652859,0.346302,9.025713
min,10230.000000,222.000000,1933.000000,1.000000,1.000000,-60.299999,4.000000,-41.299999,4.000000,902.099976,4.000000,620.400024,4.000000,0.000000,4.000000,0.000000,4.000000,1.000000,4.100000,-65.199997,0.000000,0.400000
25%,233410.000000,99999.000000,1977.000000,3.000000,8.000000,37.799999,7.000000,28.900000,6.000000,1009.500000,6.000000,945.900024,6.000000,6.300000,6.000000,3.400000,7.000000,7.800000,20.000000,30.200001,0.000000,3.800000
50%,514630.000000,99999.000000,1989.000000,6.000000,16.000000,55.000000,8.000000,43.900002,8.000000,1014.500000,8.000000,992.900024,8.000000,9.700000,8.000000,5.800000,8.000000,11.100000,25.100000,46.000000,0.000000,7.500000
75%,724825.500000,99999.000000,2000.000000,10.000000,23.000000,70.149998,23.000000,56.574999,23.000000,1020.700012,19.000000,1008.700012,23.000000,14.625000,23.000000,9.000000,23.000000,15.500000,30.950000,60.099998,0.010000,15.475000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,104.199997,24.000000,80.400002,24.000000,1056.800049,24.000000,1030.599976,24.000000,99.400002,24.000000,63.900002,24.000000,64.900002,52.400002,84.199997,7.330000,42.500000


### View the first 10 rows

In [12]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,130140,99999,1979,12,20,32.400002,24,31.299999,24.0,NaN,NaN,NaN,NaN,1.000000,24.0,1.5,24.0,4.9,NaN,30.200001,False,None,None,0.83,7.1,True,True,True,True,True,True
1,933370,99999,1986,3,9,57.700001,5,45.299999,5.0,NaN,NaN,NaN,NaN,9.900000,5.0,5.6,5.0,7.0,NaN,42.299999,True,None,None,0.00,NaN,True,True,True,True,True,True
2,702490,99999,1989,6,25,50.400002,8,37.799999,8.0,NaN,NaN,NaN,NaN,24.200001,8.0,11.5,8.0,21.0,32.099998,45.000000,True,None,None,NaN,NaN,False,False,False,False,False,False
3,526570,99999,1988,12,3,20.700001,4,2.800000,4.0,NaN,NaN,732.200012,4.0,18.600000,4.0,5.8,4.0,13.6,NaN,7.200000,True,None,None,0.00,NaN,False,False,False,False,False,False
4,109711,99999,1969,6,27,58.099998,15,50.000000,15.0,NaN,NaN,939.299988,5.0,5.100000,15.0,1.9,15.0,6.0,NaN,48.400002,True,None,None,NaN,NaN,True,True,True,True,True,True
5,403560,99999,2002,3,15,57.000000,24,38.200001,24.0,NaN,NaN,916.000000,8.0,6.200000,24.0,5.2,23.0,9.9,NaN,44.599998,False,None,None,0.00,NaN,False,False,False,False,False,False
6,578720,99999,1979,12,31,47.799999,4,42.799999,4.0,1018.500000,4.0,NaN,NaN,6.100000,4.0,9.7,4.0,11.7,NaN,39.200001,True,None,None,0.00,NaN,False,False,False,False,False,False
7,150730,99999,1990,7,13,62.200001,13,59.099998,4.0,1017.400024,4.0,972.400024,4.0,9.300000,12.0,0.0,12.0,NaN,NaN,34.200001,True,None,None,0.00,NaN,False,False,False,False,False,False
8,360910,99999,1985,10,21,26.299999,6,19.600000,6.0,1031.099976,6.0,NaN,NaN,26.900000,6.0,0.6,6.0,1.9,NaN,13.600000,True,None,None,0.12,1.6,False,False,False,False,False,False
9,289090,99999,1969,6,8,72.099998,7,50.700001,7.0,1011.500000,7.0,NaN,NaN,6.200000,7.0,8.3,7.0,15.9,NaN,59.000000,False,None,None,0.00,NaN,False,False,False,False,False,False


In [13]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
746,430140,1992,9,3,7.33
1323,913170,1983,8,3,4.21
861,619800,2008,1,30,4.13
841,478130,1956,8,27,3.98
515,22880,1978,1,9,3.94
1342,577130,1983,4,15,3.39
374,334290,1963,12,18,2.95
1641,479090,2007,8,12,2.46
1798,299150,1980,6,8,2.36
1081,432790,1999,8,21,2.36


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [14]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


We are ready for reading the data from BigQuery. Lets import libraries.

In [0]:
import tensorflow as tf
from tensorflow_io.bigquery import BigQueryClient

You can find out more about [Google Public Dataset](https://cloud.google.com/bigquery/public-data "public dataset"). We are going to use `wikipedia` table.</br> 
You need to know the column names and correct data types to extract the data. You can see the schema information in table [Wikipedia](https://console.cloud.google.com/bigquery?_ga=2.134866614.610124587.1588637836-1349673052.1577980292&project=project1-190517&folder=&organizationId=&p=bigquery-public-data&d=samples&t=wikipedia&page=table "Wikipedia summary table schema")

In [0]:
PROJECT_ID = "project1-190517" # This is from what you created in your Google Cloud Account. 
DATASET_GCP_PROJECT_ID = "bigquery-public-data"
DATASET_ID = "samples"
TABLE_ID = "wikipedia"

In [0]:
# Use this function to create a BigQuery client with read session.
def run_bqsession(num_iterations):
  batch_size = 2048
  client = BigQueryClient()
  read_session = client.read_session(
      "projects/" + PROJECT_ID,
      DATASET_GCP_PROJECT_ID, TABLE_ID, DATASET_ID,
      ["title",
       "id",
       "num_characters",
       "language",
       "timestamp",
       "wp_namespace",
       "contributor_username"],
      [tf.string,
       tf.int64,
       tf.int64,
       tf.string,
       tf.int64,
       tf.int64,
       tf.string],
      requested_streams=10
  )

  dataset = read_session.parallel_read_rows(sloppy=True).batch(batch_size)
  return dataset


  

In [0]:
batcheddataset = run_bqsession(1000)

Check the type of `batcheddataset`.

In [32]:
type(batcheddataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

Check its result:

In [33]:
next(iter(batcheddataset))

OrderedDict([('contributor_username',
              <tf.Tensor: shape=(2048,), dtype=string, numpy=
              array([b'', b'Quuxplusone', b'', ..., b'SergioGeorgini', b'', b''],
                    dtype=object)>),
             ('id',
              <tf.Tensor: shape=(2048,), dtype=int64, numpy=array([ 1462053, 20588767,  1334066, ...,  8763706, 19734410,  3934010])>),
             ('language',
              <tf.Tensor: shape=(2048,), dtype=string, numpy=array([b'', b'', b'', ..., b'', b'', b''], dtype=object)>),
             ('num_characters',
              <tf.Tensor: shape=(2048,), dtype=int64, numpy=array([20009,   702, 22598, ...,  6866,  6404, 31562])>),
             ('timestamp', <tf.Tensor: shape=(2048,), dtype=int64, numpy=
              array([1173977859, 1228703659, 1233678081, ..., 1172320333, 1231434952,
                     1224832272])>),
             ('title', <tf.Tensor: shape=(2048,), dtype=string, numpy=
              array([b'Strait of Messina Bridge', b'File tal

### Examine output

Lets print out first five records in this dataset.

In [34]:
n = 0
for next_element in batcheddataset:
  n+=1
  if (n < 6):
    tf.print(next_element)
  else:
    break

OrderedDict([('contributor_username', ["" "Quuxplusone" "" ... "SergioGeorgini" "" ""]),
             ('id', [1462053 20588767 1334066 ... 8763706 19734410 3934010]),
             ('language', ["" "" "" ... "" "" ""]),
             ('num_characters', [20009 702 22598 ... 6866 6404 31562]),
             ('timestamp', [1173977859 1228703659 1233678081 ... 1172320333 1231434952 1224832272]),
             ('title', ["Strait of Messina Bridge" "File talk:Quigley down under.jpg" "Humayun Ahmed" ... "Template talk:Infobox Disney ride" "Friends, Lovers, Brothers, and Others" "History of beer"]),
             ('wp_namespace', [0 7 0 ... 11 0 0])])
OrderedDict([('contributor_username', ["" "" "" ... "" "" "Hiding"]),
             ('id', [16100029 11802780 5710507 ... 28320 184170 3934822]),
             ('language', ["" "" "" ... "" "" ""]),
             ('num_characters', [29658 10255 36177 ... 20757 8221 23982]),
             ('timestamp', [1202059478 1235735112 1128136733 ... 1237945853 11362

Now we have `batcheddataset` as a `TensorFlow` iterator. It is one of the commonly used entry point data types for model training.